In [13]:
import os

from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("GEMINI_API_KEY")

In [14]:
from langchain_google_genai import ChatGoogleGenerativeAI 

In [15]:
llm = ChatGoogleGenerativeAI(
    api_key = api_key,
    model = "gemini-2.0-flash"
)

In [ ]:
import os
import re
from datetime import datetime, timedelta
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.chat_message_histories import ChatMessageHistory
import dateparser

load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

llm = ChatGoogleGenerativeAI(
    api_key=api_key,
    model="gemini-2.0-flash"
)

appointment_pattern = {
    "user_name": r"(?:name[:\s]+)([A-Za-z\s]+)|(?:my name is\s+)([A-Za-z\s]+)|(?:i am\s+)([A-Za-z\s]+)",
    "date": r"(?:date[:\s]+)([A-Za-z0-9\s,]+)|(?:on\s+)([A-Za-z0-9\s,]+)",
    "time": r"(?:time[:\s]+)([0-9:]+\s*(?:AM|PM|am|pm)?)|(?:at\s+)([0-9:]+\s*(?:AM|PM|am|pm)?)",
    "email": r"(?:email[:\s]+)([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)|([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)",
    "appointment_type": r"(?:appointment_type[:\s]+)(Data Science|AI\/ML|Application Development|Database Development)|(?:for\s+)(Data Science|AI\/ML|Application Development|Database Development)|(?:regarding\s+)(Data Science|AI\/ML|Application Development|Database Development)"
}

system_prompt = """
You are an expert and helpful assistant. Your responsibility is to Book Appointment with "Sabir".

Extract relevant details from the user's prompt. You should collect information sequentially if not provided all at once:
1. user_name - Ask for their name if not provided
2. date - Ask for their preferred date if not provided
3. time - Ask for their preferred time if not provided
4. email - Ask for their email address if not provided
5. appointment_type - Ask them to choose from: Data Science, AI/ML, Application Development, Database Development

Once all details are collected, confirm the appointment details with the user and respond with a nice confirmation message.

Remember to only book appointments with Sabir who specializes in Data Science, AI/ML, Application Development, and Database Development.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
])

def extract_appointment_details(user_input):
    appointment_details = {}
    
    for field, pattern in appointment_pattern.items():
        match = re.search(pattern, user_input, re.IGNORECASE)
        if match:
            caught_group = next((g for g in match.groups() if g is not None), None)
            if caught_group:
                appointment_details[field] = caught_group.strip()
    
    if "date" not in appointment_details:
        try:
            dates = parse_dates(user_input)
            if dates:
                appointment_details["date"] = dates[0].strftime("%Y-%m-%d")
        except:
            pass
    
    return appointment_details

def format_time(time_str):
    try:
        time_str = time_str.strip().lower()
        
        formats = [
            "%I:%M %p",
            "%I:%M%p",
            "%H:%M",
            "%I %p"      
        ]
        
        for fmt in formats:
            try:
                time_obj = datetime.strptime(time_str, fmt)
                return time_obj.strftime("%H:%M")
            except ValueError:
                continue
        
        if ":" not in time_str:
            match = re.match(r"(\d+)\s*(am|pm)", time_str)
            if match:
                hour, meridiem = match.groups()
                hour = int(hour)
                if meridiem == "pm" and hour < 12:
                    hour += 12
                elif meridiem == "am" and hour == 12:
                    hour = 0
                return f"{hour:02d}:00"
        
        return time_str
    except Exception as e:
        print(f"Error formatting time: {e}")
        return time_str

def save_appointment(details, message_history):
    if not details:
        return "No appointment details found to save."
    
    if 'time' in details:
        try:
            details['time'] = format_time(details['time'])
        except Exception as e:
            print(f"Error formatting time: {e}")
    
    appointment_str = "APPOINTMENT_DETAILS: " + ", ".join([f"{k}: {v}" for k, v in details.items()])
    
    message_history.add_user_message("Save my appointment")
    message_history.add_ai_message(appointment_str)
    
    return details

def has_all_required_details(details):
    required_fields = ["user_name", "date", "time", "email", "appointment_type"]
    return all(field in details for field in required_fields)

def retrieve_appointments(message_history, query=None):
    appointments = []
    
    messages = message_history.messages
    
    for message in messages:
        if hasattr(message, 'content') and "APPOINTMENT_DETAILS:" in message.content:
            appointment_info = message.content.split("APPOINTMENT_DETAILS:")[1].strip()
            appointments.append(appointment_info)
    
    if not appointments:
        return "No appointments found."
    
    if query:
        filtered = []
        for appointment in appointments:
            if query.lower() in appointment.lower():
                filtered.append(appointment)
        
        if filtered:
            return "Found these appointments matching your query:\n" + "\n".join(filtered)
        else:
            return "No appointments found matching your query."
    
    return "Here are your appointments:\n" + "\n".join(appointments)

def is_booking_request(user_input):
    booking_keywords = ["book", "schedule", "make", "set", "create", "arrange", "appointment"]
    return any(keyword in user_input.lower() for keyword in booking_keywords)

def is_retrieval_request(user_input):
    retrieval_keywords = ["show", "get", "find", "view", "check", "see", "tell", "when is", "do i have"]
    appointment_keywords = ["appointment", "schedule", "booking"]
    
    has_retrieval = any(keyword in user_input.lower() for keyword in retrieval_keywords)
    has_appointment = any(keyword in user_input.lower() for keyword in appointment_keywords)
    
    return has_retrieval and has_appointment

def get_next_question(details):
    if "user_name" not in details:
        return "Could you please tell me your name?"
    elif "date" not in details:
        return "On what date would you like to schedule your appointment?"
    elif "time" not in details:
        return "What time would work best for you?"
    elif "email" not in details:
        return "Please provide your email address for the appointment confirmation."
    elif "appointment_type" not in details:
        return "What type of service are you interested in? Sabir specializes in: Data Science, AI/ML, Application Development, or Database Development."
    else:
        return None

conversation_states = {}

def process_user_input(user_input, session_id="default"):
    if session_id not in conversation_states:
        conversation_states[session_id] = {
            "message_history": ChatMessageHistory(),
            "current_details": {},
            "collecting_info": False
        }
    
    state = conversation_states[session_id]
    message_history = state["message_history"]
    current_details = state["current_details"]
    
    new_details = extract_appointment_details(user_input)
    
    current_details.update(new_details)
    
    if is_booking_request(user_input) or state["collecting_info"]:
        state["collecting_info"] = True
        
        if has_all_required_details(current_details):
            save_appointment(current_details, message_history)
            
            confirmation = f"""
Great! I've booked your appointment with Sabir.

Appointment Details:
- Name: {current_details['user_name']}
- Date: {current_details['date']}
- Time: {current_details['time']}
- Email: {current_details['email']}
- Service: {current_details['appointment_type']}

Thank you for scheduling with us. You will receive a confirmation email shortly.
Is there anything else I can help you with?
"""
            state["collecting_info"] = False
            state["current_details"] = {}
            
            message_history.add_user_message(user_input)
            message_history.add_ai_message(confirmation)
            
            return confirmation
        else:
            next_question = get_next_question(current_details)
            message_history.add_user_message(user_input)
            message_history.add_ai_message(next_question)
            
            return next_question
    
    elif is_retrieval_request(user_input):
        response = retrieve_appointments(message_history, user_input)
        
        message_history.add_user_message(user_input)
        message_history.add_ai_message(response)
        
        return response
    
    else:
        chain = prompt | llm | StrOutputParser()
        
        chain_with_history = RunnableWithMessageHistory(
            chain,
            lambda session_id: message_history,
            input_messages_key="input",
            history_messages_key="history",
        )
        
        response = chain_with_history.invoke(
            {"input": user_input},
            config={"configurable": {"session_id": session_id}}
        )
        
        return response

if __name__ == "__main__":
    print("Welcome to the Appointment Booking System with Sabir!")
    print("You can book appointments for Data Science, AI/ML, Application Development, or Database Development.")
    print("Type 'exit' to quit.\n")
    
    session_id = f"session_{datetime.now().strftime('%Y%m%d%H%M%S')}"
    
    while True:
        user_input = input("You: ")
        
        if user_input.lower() == "exit":
            print("Thank you for using the Appointment Booking System. Goodbye!")
            break
        
        response = process_user_input(user_input, session_id)
        print(f"Bot: {response}")

Welcome to the Appointment Booking System with Sabir!
You can book appointments for Data Science, AI/ML, Application Development, or Database Development.
Type 'exit' to quit.

Bot: Okay Fahim, I can help you with that.

To book an appointment with Sabir, I need a few more details.

What date would you like to book your appointment?
Bot: Okay, Fahim. And what time would you like to book your appointment on March 20, 2025?
Bot: Okay, Fahim. I'll also need your email address.
Bot: That doesn't seem to be a valid email address. Can you please provide a valid email address?
Bot: Okay, Fahim. Lastly, what type of appointment would you like to book with Sabir? Please choose from the following options:

*   Data Science
*   AI/ML
*   Application Development
*   Database Development
Bot: Okay Fahim, let me confirm the details.

You would like to book an appointment with Sabir for Data Science on March 20, 2025, at 9am with the email address fahimsabir@gmail.com. Is that correct?
Bot: Great! Yo